In [15]:
import random
from faker import Faker
from tqdm import tqdm
from datetime import time, datetime

In [2]:
habitaciones = {}
tipos_prices = {1: 350000, 2: 250000, 3: 450000, 4: 200000, 5: 100000}
tipos_capacidad = {1: 2, 2: 3, 3: 6, 4: 4, 5: 2}
room_ids = []

# Function to generate random uppercase letter
def random_letter():
    return random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

# Function to generate a random 3-digit number
def random_number():
    return random.randint(100, 999)

# Function to generate a random integer between 1 and 5 (inclusive)
def random_tipo():
    return random.randint(1, 5)

# Generating SQL INSERT statements for habitaciones table
with open('populate_habitaciones.sql', 'w') as file:
    for i in range(1000):  # Generating 500 records
        
        while True:
            room_id = random_letter() + str(random_number())  # Generating id
            if room_id not in room_ids:
                room_ids.append(room_id)
                break
            
        tipo = random_tipo()  # Generating tipo
        habitaciones[room_id] = tipo  # Storing id and tipo in dictionary
        insert_statement = f"('{room_id}', {tipo});"
        insert_statement += "\n"
        file.write("INSERT INTO habitaciones (id, tipo) VALUES " + insert_statement)

In [3]:
usuarios_ids = []

fake = Faker()

def random_user_type():
    return random.choice([111, 222, 333, 444, 555])

def random_name():
    return fake.name()

def random_email():
    return fake.email()

def random_password():
    return fake.lexify(text='????????')

def random_num_doc():
    return fake.random_number(8)

def random_tipo_doc():
    return random.choice(['CC', 'PA', 'CE'])

user_ids = []

with open('populate_usuarios.sql', 'w') as file:
    for i in tqdm(range(150000)):  # Generating 150000 records
        
        while True:
            num_doc = random_num_doc()
            tipo_doc = random_tipo_doc()
            if (num_doc, tipo_doc) not in user_ids:
                user_ids.append((num_doc, tipo_doc))
                break
            
        tipo = random_user_type()
        nombre = random_name()
        email = random_email()
        contrasenia = random_password()
        insert_statement = f"({num_doc}, '{tipo_doc}', {tipo}, '{nombre}', '{email}', '{contrasenia}');"
        insert_statement += "\n"
        file.write("INSERT INTO usuarios (num_doc, tipo_doc, tipo, nombre, email, contrasenia) VALUES " + insert_statement)

100%|██████████| 150000/150000 [07:33<00:00, 330.40it/s]


In [33]:
def random_id_servicio():
    return random.randint(70, 101)

def random_usr_id():
    return random.choice(user_ids)

def random_room_id():
    return random.choice(room_ids)

def random_date(initial_date='01/01/2010', final_date='11/07/2013'):
    #date format dd/mm/yyyy, example: 01/01/2010
    return fake.date_between(start_date=datetime.strptime(initial_date, '%d/%m/%Y'), end_date=datetime.strptime(final_date, '%d/%m/%Y')).strftime('%d/%m/%Y')

def random_time():
    return fake.time(pattern="%H:%M")

def random_time_greater_than(t):
    
    given_hour = int(t.split(':')[0])
    given_minute = int(t.split(':')[1])
    if given_hour == 23:
        random_hour = given_hour
        random_minute = random.randint(given_minute, 59)
    else:
        random_hour = random.randint(given_hour + 1, 23)
        random_minute = random.randint(0, 59)
    return time(random_hour, random_minute)

def random_costo_servicio():
    return random.randint(10000, 200000)

with open('populate_reservas_servicio.sql', 'w') as file:
    for i in tqdm(range(100000)):  # Generating 100000 records
        id_habitacion = random_room_id()
        id_servicio = random_id_servicio()
        id_consumidor, tipo_id_consumidor = random_usr_id()
        fecha = random_date()
        hora_inicio = random_time()
        hora_fin = random_time_greater_than(hora_inicio)
        costo = random_costo_servicio()
        
        insert_statement = f"('{id_habitacion}', {id_servicio}, {id_consumidor}, '{tipo_id_consumidor}', TO_DATE('{fecha}', 'DD/MM/YYYY'), '{hora_inicio}', '{hora_fin}', {costo});"
        insert_statement += "\n"
        file.write("INSERT INTO reservas_servicio (id_habitacion, id_servicio, id_consumidor, tipo_id_consumidor, fecha, hora_inicio, hora_fin, costo) VALUES " + insert_statement)

100%|██████████| 100000/100000 [00:06<00:00, 15944.30it/s]


In [34]:
ids_cuentas_estadia = []

with open('populate_cuentas_estadia.sql', 'w') as file:
    for i in tqdm(range(100000)):  # Generating 100000 records
        while True:
            id = random.randint(1, 1000000)
            if id not in ids_cuentas_estadia:
                ids_cuentas_estadia.append(id)
                break
        
        id_habitacion = random_room_id()
        monto = random.randint(100000, 2000000)
        abierta = random.choice([0, 1])
        
        insert_statement = f"({id}, '{id_habitacion}', {monto}, {abierta});"
        insert_statement += "\n"
        file.write("INSERT INTO cuentas_estadia (id, id_habitacion, monto, abierta) VALUES " + insert_statement)

100%|██████████| 100000/100000 [00:31<00:00, 3223.44it/s]


In [35]:
cuentas_en_reservas = []
def days_between_dates(start_date_str, end_date_str):
    date_format = "%d/%m/%Y"  # Specify the format of the date strings

    # Convert the strings to datetime objects
    start_date = datetime.strptime(start_date_str, date_format)
    end_date = datetime.strptime(end_date_str, date_format)

    # Calculate the difference in days
    difference = end_date - start_date

    # Return the number of days (using .days attribute)
    return difference.days

with open('populate_reservas_habitacion.sql', 'w') as file:
    for i in tqdm(range(300000)):  # Generating 300000 records        
        id_habitacion = random_room_id()
        num_doc, tipo_doc = random_usr_id()
        fecha_entrada = random_date()
        fecha_salida = random_date(initial_date=fecha_entrada, final_date='11/07/2023')
        plan_consumo = random.choice([12, 34, 56, 78])
        num_personas = tipos_capacidad[habitaciones[id_habitacion]]
        costo = tipos_prices[habitaciones[id_habitacion]] * days_between_dates(fecha_entrada, fecha_salida)
        
        id_cuenta = random.choice(ids_cuentas_estadia)
                
        insert_statement = f"('{id_habitacion}', {num_doc}, '{tipo_doc}', TO_DATE('{fecha_entrada}', 'DD/MM/YYYY'), TO_DATE('{fecha_salida}', 'DD/MM/YYYY'), {plan_consumo}, {num_personas}, {costo}, {id_cuenta});"
        insert_statement += "\n"
        file.write("INSERT INTO reservas_habitacion (id_habitacion, num_doc, tipo_doc, fecha_entrada, fecha_salida, plan_consumo, num_personas, costo, id_cuenta) VALUES " + insert_statement)

  0%|          | 0/300000 [00:00<?, ?it/s]

100%|██████████| 300000/300000 [00:29<00:00, 10300.27it/s]


In [40]:
ids_check_ins = []
fechas_check_ins = {}

with open('populate_check_ins.sql', 'w') as file:
    for i in tqdm(range(100000)):        
        
        while True:
            id = random.randint(1, 1000000)
            if id not in ids_check_ins:
                ids_check_ins.append(id)
                break
        id_habitacion = random_room_id()
        num_doc, tipo_doc = random_usr_id()
        fecha = random_date()
        fechas_check_ins[id] = fecha
        hora = random_time()
                
        insert_statement = f"({id}, '{id_habitacion}', {num_doc}, '{tipo_doc}', TO_DATE('{fecha}', 'DD/MM/YYYY'), '{hora}');"
        insert_statement += "\n"
        file.write("INSERT INTO check_ins (id, id_habitacion, num_doc, tipo_doc, fecha, hora) VALUES " + insert_statement)

100%|██████████| 100000/100000 [00:34<00:00, 2871.29it/s]


In [41]:
with open('populate_check_outs.sql', 'w') as file:
    for i in tqdm(range(100000-1)):        
        
        id_check_in = random.choice(ids_check_ins)
        ids_check_ins.remove(id_check_in)
        
        fecha = random_date(initial_date=fechas_check_ins[id_check_in], final_date='11/07/2023')
        hora = random_time()
        ingresos_totales = random.randint(100000, 2000000)
                
        insert_statement = f"({id_check_in}, 'TO_DATE('{fecha}', 'DD/MM/YYYY'), '{hora}', {ingresos_totales});"
        insert_statement += "\n"
        file.write("INSERT INTO check_outs (id_check_in, fecha, hora, ingresos_totales) VALUES " + insert_statement)

100%|██████████| 99999/99999 [00:27<00:00, 3675.99it/s] 
